## 28. プログレッシブGAN
**プログレッシブGAN(PGGAN/ProGAN)**はフルHDの写真画質画像を生成することに成功した技術である。  
プログレッシブGANの特徴には以下のものがある。  
- プログレッシブにネットワークを成長させ、高解像度層へ滑らかに移行する
- ミニバッチ標準偏差
- 学習率の平滑化
- ピクセルごとの特徴正規化  
  
プログレッシブGANにおいても、潜在空間は意味のある特徴を表している。  
これにより例えば眼鏡を追加する差分ベクトルを見つけることができれば、別の顔画像にも眼鏡をかけるようなことができる。  
また、1つ目のベクトルが表す顔画像から、2つ目の顔画像に、徐々に滑らかに変化する顔画像を得ることができる。  
  
#### プログレッシブな成長
まずは少数の低解像度の畳み込み層から始めて、訓練が進むにつれてだんだん多くの高解像度層に移行していく手法のこと。  
このとき、いきなり高解像度に飛び込むのではなく、0と1の間をとるパラメータ$\alpha$を使って、少しずつ新しい解像度の層を入れていく。  
$\alpha$は、古い層をそのまま拡大したものと、新しい高解像度層との混ぜ合わせた比率である。

In [2]:
import tensorflow as tf
import tensorflow.keras as K

def upscale_layer(layer, upscale_factor):
    """
    テンソル：[group, height, width, channels]
    """
    height, width = layer.get_shape()[1:3]
    size = (upscale_factor * height, upscale_factor * width)
    upscale_layer = tf.image.resize_nearest_neighbor(layer, size)
    return upscale_layer

def smoothly_merge_last_layer(list_of_layers, alpha):
    last_fully_trained_layer = list_of_layres[-2]
    last_layer_upscaled = upscale_layer(last_fully_trained_layer, 2)
    
    larger_native_layer = list_of_layeres[-1]
    
    assert larger_native_layer.get_shape() == last_layer_upscaled.get_shape()
    
    new_layer = (1-alpha)*upscale_layer + larger_native_layer*alpha
    return new_layer

#### ミニバッチ標準偏差
サンプルが十分ばらけているかを調べるため、識別機に1つだけ統計量を入力する。  
この統計量は、生成器が生成した画像や、本物のデータから来たミニバッチ内の全画素値の標準誤差である。  
もし、識別器があるバッチからくる画素の標準誤差が、本物のデータに比べて小さければ、偽物であると判断できる。  
従って、生成器は識別器をだますためには生成されるサンプルのばらつきを増やさざるを得なくなる。  
ステップとしては以下のようになる。  
  
1. \[4Dから3D\]バッチ内のすべての画像、その他すべてのチャンネル(幅、高さ、色)に渡って、標準偏差を計算する。  
この結果、各ピクセル、各色チャンネルに標準偏差が格納された、1枚の画像が出来上がる。  
2. \[3Dから2D\]全ての色チャンネルの標準偏差の平均をとって、各画素がその平均値を格納している特徴マップ(行列)を得る。  
3. \[2Dから単一の値/0D\]行列の全画素に格納された標準偏差値の平均をとり、単一のスカラ値にする。

In [3]:
def minibatch_std_layer(layer, group_size=4):
    group_size = K.backend.minimum(group_size, tf.shape(layer)[0])
    
    shape = list(K.int_shape(input))
    shape[0] = tf.shape(input)[0]
    
    minibatch = K.backend.reshape(layer, (group_size, -1, shape[1], shape[2], shape[3]))
    
    minibatch -= tf.reduce_mean(minibatch, axis=0, keepdims=True)
    minibatch = tf.reduce_mean(K.backend.square(minibatch), axis=0)
    minibatch = K.backend.square(minibatch+1e8)
    minibatch = tf.reduce_mean(minibatch, axis=[1,2,4], keepdims=True)
    minibatch = K.backend.tile(minibatch, [group_size, 1, shape[2], shape[3]])
    
    return L.backend.concatenate([layer, minibatch], axis=1)

#### 学習率の平滑化
なぜ、うまく動作するかはわかっていない。  

In [4]:
def equalize_learning_rate(shape, gain, fan_in=None):
    if fan_in is None: fan_in = np.prod(shape[:-1])
    
    std = gain / K.sqrt(fan_in)
    
    wscale = K.constant(std, name='wscale', dtype=np.float32)
    
    adjusted_weights = K.get_value('layer', shape=shape, initializer=tf.initializers.random_normal()) * wscale
    
    return adjusted_weights

#### ピクセルごとの特徴正規化
訓練の安定化のために、正規化が使用される。  
普通のバッチ正規化を使わないのは、解像度が高くなるとメモリをあまりに消費するということがある。

In [5]:
def pixelwise_feat_norm(inputs, **kwargs):
    normalization_constant = K.backend.sqrt(K.backend.mean(inputs**2, axis=-1, keepdims=True) + 1.0e-8)
    return inputs / normalization_constant

#### tensorflw Hub

In [7]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub

with tf.Graph().as_default():
    # import the progressive GAN from TFHub
    module = hub.Module("https://tfhub.dev/google/progan-128/1")
    # latent dimension that gets 
    latent_dim = 512

    # Change the seed to get different faces.
    latent_vector = tf.random_normal([1, latent_dim], seed=1337)

    # Uses module to generate images from the latent space.
    interpolated_images = module(latent_vector)

    # runs the tensorflow session and gets back the image in shape (1,128,128,3)
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        image_out = session.run(interpolated_images)

plt.imshow(image_out.reshape(128,128,3))
plt.show()

KeyboardInterrupt: 

In [12]:
#@title Imports and function definitions

import imageio
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import time
from IPython import display
from skimage import transform

# We could retrieve this value from module.get_input_shapes() if we didn't know
# beforehand which module we will be using.
latent_dim = 512


# Interpolates between two vectors that are non-zero and don't both lie on a
# line going through origin. First normalizes v2 to have the same norm as v1. 
# Then interpolates between the two vectors on the hypersphere.
def interpolate_hypersphere(v1, v2, num_steps):
    v1_norm = tf.norm(v1)
    v2_norm = tf.norm(v2)
    v2_normalized = v2 * (v1_norm / v2_norm)

    vectors = []
    for step in range(num_steps):
        interpolated = v1 + (v2_normalized - v1) * step / (num_steps - 1)
        interpolated_norm = tf.norm(interpolated)
        interpolated_normalized = interpolated * (v1_norm / interpolated_norm)
        vectors.append(interpolated_normalized)
    return tf.stack(vectors)


# Given a set of images, show an animation.
def animate(images):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images)
    with open('./animation.gif','rb') as f:
        display.display(display.Image(data=f.read(), height=300))


# Simple way to display an image.
def display_image(image):
    plt.figure()
    plt.axis("off")
    plt.imshow(image)


# Display multiple images in the same figure.
def display_images(images, captions=None):
    num_horizontally = 5
    f, axes = plt.subplots(
        len(images) // num_horizontally, num_horizontally, figsize=(20, 20))
    for i in range(len(images)):
        axes[i // num_horizontally, i % num_horizontally].axis("off")
        if captions is not None:
            axes[i // num_horizontally, i % num_horizontally].text(0, -3, captions[i])
        axes[i // num_horizontally, i % num_horizontally].imshow(images[i])
    f.tight_layout()


#tf.logging.set_verbosity(tf.logging.ERROR)

In [14]:
def interpolate_between_vectors():
    with tf.Graph().as_default():
        module = hub.Module("https://tfhub.dev/google/progan-128/1")

        # Change the seed to get different random vectors.
        v1 = tf.random_normal([latent_dim], seed=3)
        v2 = tf.random_normal([latent_dim], seed=1)
    
        # Creates a tensor with 50 steps of interpolation between v1 and v2.
        vectors = interpolate_hypersphere(v1, v2, 25)

        # Uses module to generate images from the latent space.
        interpolated_images = module(vectors)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        interpolated_images_out = session.run(interpolated_images)

    animate(interpolated_images_out)

interpolate_between_vectors()

KeyboardInterrupt: 

In [ ]:
image_from_module_space = True  # @param { isTemplate:true, type:"boolean" }

def get_module_space_image():
    with tf.Graph().as_default():
        module = hub.Module("https://tfhub.dev/google/progan-128/1")
        vector = tf.random_normal([1, latent_dim], seed=4)
        images = module(vector)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        image_out = session.run(images)[0]
    return image_out

def upload_image():
    uploaded = files.upload()
    image = imageio.imread(uploaded[uploaded.keys()[0]])
    return transform.resize(image, [128, 128])

if image_from_module_space:
    target_image = get_module_space_image()
else:
    target_image = upload_image()
display_image(target_image)

In [ ]:
def find_closest_latent_vector(num_optimization_steps):
    images = []
    losses = []
    with tf.Graph().as_default():
        module = hub.Module("https://tfhub.dev/google/progan-128/1")

        initial_vector = tf.random_normal([1, latent_dim], seed=5)

        vector = tf.get_variable("vector", initializer=initial_vector)
        image = module(vector)

        target_image_difference = tf.reduce_sum(
            tf.losses.absolute_difference(image[0], target_image[:,:,:3]))

        # The latent vectors were sampled from a normal distribution. We can get
        # more realistic images if we regularize the length of the latent vector to 
        # the average length of vector from this distribution.
        regularizer = tf.abs(tf.norm(vector) - np.sqrt(latent_dim))
    
        loss = target_image_difference + regularizer
    
        optimizer = tf.train.AdamOptimizer(learning_rate=0.3)
        train = optimizer.minimize(loss)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        for _ in range(num_optimization_steps):
            _, loss_out, im_out = session.run([train, loss, image])
            images.append(im_out[0])
            losses.append(loss_out)
            print(loss_out)
    return images, losses


result = find_closest_latent_vector(num_optimization_steps=40)

In [ ]:
captions = [ f'Loss: {l:.2}' for l in result[1]]
display_images(result[0], captions)